In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    #print(detections.shape)
    
    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
        
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
        
            
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return preds

In [3]:
prototxtPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\deploy.prototxt"
weightsPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model(r"C:\Users\vasu2\Desktop\Mask Detection Test\maskdata\MaskDetectionModel")

#vs = VideoStream(src=0).start()

cam = cv2.VideoCapture(0)

# while True:
#     frame = vs.read()
#     frame = imutils.resize(frame, width=400)
#     assert not isinstance(frame,type(None)), 'frame not found'
#     (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

#     for (box, pred) in zip(locs, preds):
#         (startX, startY, endX, endY) = box
#         (mask, withoutMask) = pred

#         label = "Mask" if mask > withoutMask else "No Mask"
#         color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

#         label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

#         cv2.putText(frame, label, (startX, startY - 10),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
#         cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

#     cv2.imshow("Frame", frame)
#     key = cv2.waitKey(1) & 0xFF

#     if key == ord("q"):
#         break


cv2.namedWindow("Mask Detection")




while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        if(checkFaceMask(frame)):
            print ("Mask")
        else:
            print("No Mask")

        
        

cam.release()

cv2.destroyAllWindows()
#vs.stop()

Escape hit, closing...


In [4]:
def checkFaceMask(frame):
    
    prototxtPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\deploy.prototxt"
    weightsPath = r"C:\Users\vasu2\Desktop\Mask_Detection_BE\face_detector\res10_300x300_ssd_iter_140000.caffemodel"
    faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

    maskNet = load_model(r"C:\Users\vasu2\Desktop\Mask Detection Test\maskdata\MaskDetectionModel")


    frame = imutils.resize(frame, width=400)
    assert not isinstance(frame,type(None)), 'frame not found'
    preds = detect_and_predict_mask(frame, faceNet, maskNet)

    
    pred = zip(preds)
    #print (list(pred)[0][0][0])

    str = list(pred)[0][0]
    l = []
    
    print(str[0])
    print(str[1])
    #mask = list(pred)[0][0][0]
    #withoutmask = list(pred)[0][0][1]

    if str[0] > str[1]:
        return True
    else:
        return False




    #color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

    # cv2.putText(frame, label, (startX, startY - 10),
    #     cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    # cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    return True